<img src='../../img/anaconda-logo.png' align='left' style="padding:10px">
<br>
*Copyright Continuum 2012-2016 All Rights Reserved.*

# Dashboards

Dashboards are expecially useful for real-time streaming data.

In the context of doing streaming with Bokeh Server:
* Bokeh Server gives you **the ability to write call-backs** -- functions that can execute code on the server -- by writing code in python instead of javascript.
* Bokeh Server gives you a **streaming capability** -- thre is two-way data movement between the client and the server.
   * usually the server is just responding to client requests
   * in this case, the server is pushing data from the server onto the client side, to update your dashboard
   * Bokeh Server does this using Web Sockets to change and sync state.
   * `ColumnDataSource` is the mechanism that allows you to sync data between server side and client side

## Table of Contents
* [Dashboards](#Dashboards)
* [Server DashBoards](#Server-DashBoards)
	* [Inspect the source!](#Inspect-the-source!)
* [Static Dashboards](#Static-Dashboards)
	* [Set-Up](#Set-Up)
	* [Static dashboards](#Static-dashboards)
		* [Exercise: charts](#Exercise:-charts)
	* [Interactive dashboards](#Interactive-dashboards)
		* [Widgets](#Widgets)
		* [Exercise: College Scorecard](#Exercise:-College-Scorecard)
		* [Exercise: stock data](#Exercise:-stock-data)
	* [Streaming dashboards](#Streaming-dashboards)
		* [Streaming finance data](#Streaming-finance-data)
		* [Exercise: streaming](#Exercise:-streaming)


# Server DashBoards

Let's start with a DEMO!

You can create interactive dashboards with Bokeh Server. To demonstrate this, to the Terminal we go!

```bash
$ cd ~/code
$ git clone git@github.com:bokeh/bokeh.git
$ cd ~/code/bokeh/examples/app/ohlc
$ source activate iqt
$ bokeh serve ./main.py --show

# Also notice in main.py

```

## Inspect the source!

Let's open `main.py` and inspect the python source code to see how all this magic is implemented:

In [ ]:
# Uncomment the %load line below and execute this cell if you cloned the bokeh repo
# Or, even better, use a text editor to open the source file!

# %load ~/code/bokeh/examples/app/ohlc/main.py

Note the reference to `curdoc()` in the `main.py`:

```python
from bokeh.plotting import curdoc
```

This is analogous to setting `output_notebook()`

Also notice:

```python
curdoc().add_periodic_callback(update, 50)
```

For example, the `.add_periodic_callback` in `main.py` takes the following input args:

* source: `curdoc().add_periodic_callback(update, 50)`
* function: `update`
* interval: `50`, in units of milliseconds

Note that the `def update():` function calls `source.stream(new_data, 300)`, where 300 is the number of pushes that the client will maintain before it starts dropping data on the floor.

# Static Dashboards

## Set-Up

In [ ]:
from bokeh.io import output_notebook, show
output_notebook()

Any *static* Bokeh plot can be made into a dashboard with `curdoc`.

```python
from bokeh.io import curdoc

# get the data and make the plot

curdoc().add_root(plot)
```

launch the dashboard

```
$ bokeh serve --show plot.py
```

## Static dashboards

Any bokeh plot (Plotting or Chart interface) can be made into a static dash board. The plot (or layout object) must be added as the root of `curdoc()`.

### Exercise: charts

<img src='img/topics/Exercise.png' align='left' style='padding:10px'>
<br>
In a new `.py` file make a Histogram of the daily temperature in from `data/pittsburgh2013.csv`. 

Add the above code to make a dashboard and run it through the terminal.

<a href='../../../edit/Bokeh/notebooks/src/temperature-dashboard_soln.py' class='btn btn-primary'>Solution</a>

## Interactive dashboards

For dashboards that incorporate more interactivity through *widgets* only the Plotting interface is supported.

Note: It is also required to use the `ColumnDataSource` object to store and update the data.

In addiction to `curdoc()` *interactive* dashboards add:
* Widgets (slider, text fields, buttons, etc.)
* Update function
  * to update the `.data` attribute of a ColumnDataSource

```python
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider
source = ColumnDataSource(...)

slider = Slider()

def update():
    new_value = slider.value
    # compute NEW_DICT data
    source.data = NEW_DICT

slider.on_change('value',update)
update()
curdoc().add_root(plot)
```

### Widgets

http://bokeh.pydata.org/en/latest/docs/user_guide/interaction.html#button

In [ ]:
from bokeh.models.widgets import Slider
slider = Slider()
show(slider)

Sliders take
* `title`: String
* `start`: Number
* `end`: Number
* `value`: Number
* `step`: Number

In [ ]:
slider = Slider(title='years', start=2000, end=2020, value=2016, step=1)
show(slider)

In a real dashboard the `.value` will be updated based on the current position.

In [ ]:
slider.value

In [ ]:
from bokeh.models.widgets import TextInput
text = TextInput(title='Ticker', value='^GSPC')
show(text)

### Exercise: College Scorecard

<img src='img/topics/Exercise.png' align='left' style='padding:10px'>
<br><big>A template has been provided at `Bokeh/notebooks/src/college-dashboard.py`</big>

<a href='../../../edit/Bokeh/notebooks/src/college-dashboard.py' class='btn btn-primary btn-md'>Template file</a>

The script provides functions that
* read the [College Scorecard Data Set](https://collegescorecard.ed.gov/data/)
* filter the data set based on admission rate and state
* plot all of the colleges based on the filter
  * color the cost by *green-yellow-red* in increasing cost
  * scale the size of the plot by the admission rate (small circles have low admission rates)

The task is to
1. Import the tools and widgets you want to utilize
2. Preparte the tools objects
3. Prepare the widgets objects
4. Grab updated values from widgets in the `update` function
5. Register the update function for each of your widgets

<a href='../../../edit/Bokeh/notebooks/src/college-dashboard_soln.py' class='btn btn-primary'>Solution</a>

### Exercise: stock data

<img src='img/topics/Exercise.png' align='left' style='padding:10px'>
<br><big>A template has been provided at `Bokeh/notebooks/src/stocks-dashboard.py`.</big>

<a href='../../../edit/Bokeh/notebooks/src/stocks-dashboard.py' class='btn btn-primary btn-md'>Template file</a>

This script provides functions that
* download stock data from Yahoo
* compute short and long moving average windows
* identify *positions* when the moving averages cross

The task is to
1. Add widgets to change the ticker string, start and end years and the moving average windows
1. Make plot object
1. Write a function to get updated widget values and return data
1. Register the widgets with the update function

<a href='../../../edit/Bokeh/notebooks/src/stocks-dashboard_soln.py' class='btn btn-primary'>Solution</a>

## Streaming dashboards

Streaming dashboards allow data to added to the `ColumnDataSource` object on a specified interval. New data point are appended using the `stream` member method.

This function can be separate from other update functions that read updated widget values.

```python
from bokeh.io import curdoc

source = ColumnDataSource(x=[], y=[])


def stream_data():
    # get new x and y data as lists
    
    NEW_DATA = dict(
        x = new_list_x,
        y = new_list_y
    )
    source.stream(NEW_DATA)

curdoc().add_root(plot)
curdoc().add_periodic_callback(stream_data, UPDATE_INTERVAL)
```

### Streaming finance data

This is a forgotten Google API that still works.

https://web.archive.org/web/20111209142224/http://code.google.com/apis/finance/docs/finance-gadgets.html

In [ ]:
from urllib.request import urlopen

url_base = 'http://finance.google.com/finance/info?client=ig&q='

values = urlopen(url_base+'AAPL').read().decode('utf-8')
print(values)

In [ ]:
import time
import json
for _ in range(10):
    values = urlopen(url_base+'AAPL').read().decode('utf-8')
    apple = json.loads(values[3:])[0]
    print(apple['lt_dts'], apple['l'])
    time.sleep(2)

### Exercise: streaming

<img src='img/topics/Exercise.png' align='left' style='padding:10px'>
<br><big>
Using `Bokeh/notebooks/src/stocks-dashboard-live.py` write appropriate `get_stock_data` and `update_price` functions.
</big>

<a href='../../../edit/Bokeh/notebooks/src/stocks-dashboard-live.py' class='btn btn-primary btn-md'>Template file</a>

<a href='../../../edit/Bokeh/notebooks/src/stocks-dashboard-live_soln.py' class='btn btn-primary'>Solution</a>

---
*Copyright Continuum 2012-2016 All Rights Reserved.*